## 3 Different Approaches to Builidng the RAG system

### Set up

In [ ]:
''' Install dependencies '''
#pip install pypdf
#pip install langchain langchain_community langchain_core langchain_openai langchain_text_splitters
#pip install -qU langchain-chroma
#pip install langchainhub
#pip install -U "langchain[anthropic]"
#pip install -qU langchain-voyageai

In [3]:
''' Load Claude AI API key from .env '''
import os
import dotenv
from dotenv import load_dotenv
import getpass

load_dotenv()
#os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

''' Load Voyage AI API key from .env '''
os.environ["VOYAGE_API_KEY"] = os.getenv("VOYAGE_API_KEY")

''' Load Langsmith API key from .env '''
os.environ["LANGSMITH_TRACING"] = "true"
if not os.environ.get("VOYAGE_API_KEY"):
    os.environ["VOYAGE_API_KEY"] = getpass.getpass("Enter API key for Voyage AI: ")


In [11]:
''' Set up necessary libraries and environment '''
import bs4
import langchainhub
from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_anthropic import ChatAnthropic
from langchain_classic import hub

In [4]:
''' Set up document loader '''
from langchain_community.document_loaders import PyPDFLoader #alternative: PyMuPDFLoader
from langchain_community.document_loaders import PyMuPDFLoader #alternative: PyMuPDFLoader

policy_pdf = os.getenv("PDF_PATH")

loader = PyMuPDFLoader(policy_pdf, extract_tables_settings={"enabled": True})

documents = loader.load()

In [ ]:
''' Redact sensitive information from documents (PII)'''
'''
import re
from langchain_core.documents import Document

policy_number = input("Enter your policy number (used only for one-time redaction): ").strip()
redaction_patterns = {
    "email": r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b",
    "government_id": r"\b\d{6}-\d{2}-\d{4}\b",  # 123456-78-9000
}

def redact_sensitive_info(text, policy_number):
    redacted = text
    for pattern_name, pattern in redaction_patterns.items():
        redacted = re.sub(pattern, f"[Redacted_{pattern_name.upper()}]", redacted)
    redacted = re.sub(re.escape(policy_number), "[Redacted_policy_number]", redacted)
    return redacted

redacted_documents = []

for doc in documents: 
    redacted_content = redact_sensitive_info(doc.page_content, policy_number=policy_number)
    redacted_documents.append(
        Document(page_content=redacted_content))
    
documents = redacted_documents
'''

In [5]:
print("Number of pages loaded:", len(documents))
print(documents[0].page_content[:500])

Number of pages loaded: 15
1 
 AIA CI Starter Cover 
 
 9 January 2024 
AIA SINGAPORE PTE LTD – AIA CI STARTER COVER  
POLICY SCHEDULE AND POLICY CONTRACT 
 
POLICY SCHEDULE 
Group Policy Number 
: 83319 
Effective Date of Coverage :  Date of Application 
Expiry Date of Coverage 
: 6 months from effective date of coverage  
 
Table of Benefits 
 
 
Benefits 
Amount Covered (SGD) 
1. Teleconsultation Benefit via WhiteCoat 
- Consultation only (Excluding medication & delivery costs of medication)  
 As Charged, up to 2 visi


In [6]:
''' Create vector store '''

from langchain_voyageai import VoyageAIEmbeddings
from langchain_chroma import Chroma

embeddings = VoyageAIEmbeddings(model="voyage-3")

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings
    #persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

c:\Users\chery\projects\insurance-policy-rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
''' Split documents into chunks'''

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
splits = text_splitter.split_documents(documents)

print(f"Split blog post into {len(splits)} sub-documents.")

Split blog post into 59 sub-documents.


In [8]:
''' Store chunks in vector database '''

document_ids = vector_store.add_documents(documents=splits)

print(document_ids[:3])

['a24e731c-f29b-4432-9021-e7a00103de1e', '0e7cb829-96d5-4faa-95e6-207a063ee27f', '2fb221c9-4c8a-4ff5-89d8-2474a69fdc97']


In [13]:
''' Model setup '''

from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-sonnet-4-5-20250929")

### Approach 1 - RAG agent with tools

In [ ]:
''' Minimal RAG agent that wraps vector store '''

from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

''' Construct the agent with the tool and model'''

from langchain.agents import create_agent

tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from an insurance policy document. "
    "Use the tool to help answer user queries."
)
agent = create_agent(model, tools, system_prompt=prompt)

In [15]:
''' Test the agent with a simple query'''

query = (
    "What are the conditions of the critical illness benefit to be eligible for a one lump sum payout of the amount covered? \n\n"
    "Once you get the answer, look up how 'Critical Illness' is defined in the policy."
)


for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What are the conditions of the critical illness benefit to be eligible for a one lump sum payout of the amount covered? 

Once you get the answer, look up how 'Critical Illness' is defined in the policy.
================================== Ai Message ==================================

[{'text': "I'll help you find information about the critical illness benefit conditions and then look up the definition of 'Critical Illness' in the policy.", 'type': 'text'}, {'id': 'toolu_015BjR3XeyaaRPq788PA5eSB', 'input': {'query': 'conditions critical illness benefit eligible lump sum payout amount covered'}, 'name': 'retrieve_context', 'type': 'tool_use', 'caller': {'type': 'direct'}}, {'id': 'toolu_018v1bf1nFBD1XGbQHPQTJzn', 'input': {'query': 'Critical Illness definition'}, 'name': 'retrieve_context', 'type': 'tool_use', 'caller': {'type': 'direct'}}]
Tool Calls:
  retrieve_context (toolu_015BjR3XeyaaRPq788PA5eSB)
 C

### Approach 2 - RAG chain using middleware to retrieve context prior to query

In [16]:
''' Inject retrieved context into the agent's state messages using middleware '''

from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vector_store.similarity_search(last_query)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    system_message = (
        "You are a helpful assistant. Use the following context in your response:"
        f"\n\n{docs_content}"
    )

    return system_message


agent = create_agent(model, tools=[], middleware=[prompt_with_context])

In [17]:
''' Test the agent (RAG Chains) with a sample query '''

query = "What is Critical Illness Insurance?"
for step in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is Critical Illness Insurance?
================================== Ai Message ==================================

Based on the provided context, Critical Illness Insurance is a type of insurance coverage that provides financial protection when an insured person is diagnosed with or affected by a critical illness.

## Key Features:

**What it Covers:**
- Payment of a **lump sum benefit** (the Amount Covered) if the insured person:
  - Is diagnosed with a Critical Illness
  - Dies as a result of a Critical Illness
  - Undergoes surgery for a Critical Illness

**Important Conditions:**
- The diagnosis date of a Critical Illness must be made **more than 30 days** after the policy effective date or the coverage effective date (whichever is later)
- Once a claim is admitted for any of the above, the coverage terminates
- Only **one benefit** is paid - either the Critical Illness benefit OR the Death benefit

### Approach 3 - LCEL chain 

In [13]:
''' LangChain Expression Language (LCEL) chain approach '''

from langchain_anthropic import ChatAnthropic
from langchain_classic import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Create retriever to get relevan documents from vector store
retriever = vector_store.as_retriever()

# Pull RAG prompt from LangChain Hub
prompt = hub.pull("rlm/rag-prompt")

# Load Claude model for RAG chain
llm = ChatAnthropic(
    model="claude-sonnet-4-5-20250929",
    temperature=0,
    api_key=os.getenv("ANTHROPIC_API_KEY")
)

# Helper function to format retrieved documents into a string for the prompt
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain 
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

result = rag_chain.invoke("How is Critical Illness defined in the policy?")

print(result)

Critical Illness is defined as illnesses or surgical procedures that fall within the definitions and fulfill the criteria set out in the Schedule of Critical Illnesses. The diagnosis must be made more than 30 days after either the policy effective date or the insured person's coverage effective date, whichever is later. Benefits are payable when an insured person is diagnosed with, dies from, or undergoes surgery for a covered Critical Illness.


In [ ]:
vector_store.delete_collection()